In [1]:
from datasets import load_dataset
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
import re
from sklearn.metrics import classification_report

ds = load_dataset("imodels/diabetes-readmission")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/49.2M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/81410 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/20353 [00:00<?, ? examples/s]

In [2]:
df_train = pd.DataFrame(ds['train'])
df_test = pd.DataFrame(ds['test'])

In [3]:
df_train.describe()

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,change,diabetesMed,...,glyburide-metformin:Up,A1Cresult:>7,A1Cresult:>8,A1Cresult:None,A1Cresult:Norm,max_glu_serum:>200,max_glu_serum:>300,max_glu_serum:None,max_glu_serum:Norm,readmitted
count,81410.000000,81410.000000,81410.000000,81410.000000,81410.000000,81410.000000,81410.000000,81410.000000,81410.000000,81410.000000,...,81410.000000,81410.000000,81410.000000,81410.000000,81410.000000,81410.000000,81410.000000,81410.000000,81410.000000,81410.000000
mean,4.402383,43.079511,1.340400,16.020993,0.368849,0.198207,0.634922,7.424088,0.462486,0.769783,...,0.000061,0.037342,0.080875,0.832453,0.049331,0.014740,0.012455,0.947586,0.025218,0.461430
std,2.985910,19.688887,1.707738,8.128563,1.264143,0.936837,1.262987,1.931912,0.498594,0.420975,...,0.007837,0.189599,0.272644,0.373466,0.216559,0.120512,0.110908,0.222861,0.156788,0.498513
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,14.000000,132.000000,6.000000,81.000000,42.000000,76.000000,19.000000,16.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Preprocessing

In [4]:
# Function to clean column names
def clean_col_names(df):
    cols = df.columns
    new_cols = []
    for col in cols:
        new_col = col.replace('[', '').replace(']', '').replace('<', '')
        new_cols.append(new_col)
    df.columns = new_cols
    return df

In [5]:
# Example: combine 'metformin_' columns
med = "metformin"
med_cols = [c for c in df_train.columns if c.startswith(med + ":")]

df_train_cpy = df_train.copy()

# Reverse one-hot encoding (find which column is 1)
df_train_cpy[med] = df_train_cpy[med_cols].idxmax(axis=1).str.replace(med + ":", "")

# Optionally drop original one-hot columns
df_train_cpy = df_train_cpy.drop(columns=med_cols)

df_train_cpy[[med]].head()


,metformin
0,No
1,No
2,No
3,No
4,No


In [6]:
med_preproc = ["metformin", "repaglinide", "nateglinide", "chlorpropamide", "glimepiride",
               "acetohexamide", "glipizide", "glyburide", "tolbutamide", "pioglitazone", "rosiglitazone",
               "acarbose", "miglitol", "troglitazone", "tolazamide", "examide", "citoglipton",
               "insulin", "glyburide-metformin", "glipizide-metformin", "glimepiride-pioglitazone", "metformin-rosiglitazone"]

dict_med_mapping = {
    "No" : 0,
    "Down" : -1,
    "Steady" : 1,
    "Up" : 2
}

def preprocessing2(df) :
  df = df.copy() # Create a copy to avoid modifying original df_train/df_test

  # Clean column names before creating new features
  df = clean_col_names(df)

  df['labs_per_day'] = df['num_lab_procedures'] / df['time_in_hospital']
  df['meds_per_day'] = df['num_medications'] / df['time_in_hospital']
  df['out_in_ratio'] = df['number_outpatient'] / (df['number_inpatient']+1)

  toDrop_inside = ['readmitted', 'num_lab_procedures', 'time_in_hospital', 'num_medications', 'number_outpatient', 'number_inpatient']

  for med in med_preproc :
    smol_med_cols = [c for c in df.columns if c.startswith(med + ":")]
    if smol_med_cols: # Check if columns exist for this medication
      df[med] = df[smol_med_cols].idxmax(axis=1).str.replace(med + ":", "")
      # Apply the mapping to convert to numerical values
      df[med] = df[med].map(dict_med_mapping)
      toDrop_inside.extend(smol_med_cols) # Add the one-hot encoded columns to the drop list

  # Also drop the target variable 'readmitted' here
  new_df = df.drop(columns = toDrop_inside, errors='ignore') # Use errors='ignore' to avoid errors if a column is already dropped
  return new_df

df_train_cpy = preprocessing2(df_train)
df_test_cpy = preprocessing2(df_test)

## Model

In [7]:
y_train = df_train['readmitted']
y_test = df_test['readmitted']


model = xgb.XGBClassifier()
model.fit(df_train_cpy, y_train)
predictions = model.predict(df_test_cpy)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.63      0.70      0.66     11016
           1       0.59      0.52      0.55      9337

    accuracy                           0.62     20353
   macro avg       0.61      0.61      0.61     20353
weighted avg       0.61      0.62      0.61     20353

